This notebook shows how to load metadata tables from the Visual Behavior 2P Project using AllenSDK tools to find experiments of interest. 

First, we need to import libraries for plotting and manipulating data

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('notebook', font_scale=1.5, rc={'lines.markeredgewidth': 2})

In [3]:
# prefered magic functions for jupyter notebook
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
# confirm that you are currently using the newest version of SDK (2.10.0 for now)
import allensdk
allensdk.__version__

'2.15.2'

In [5]:
# import behavior projet cache class from SDK to be able to load the data
import allensdk.brain_observatory.behavior.behavior_project_cache as bpc

## Load data tables 

This code block allows you to use ```behavior_project_cache``` (bpc) class to get behavior and ophys tables.

In [6]:

my_cache_dir = r'\Data\visual_behavior_ophys_cache_dir'

bc = bpc.VisualBehaviorOphysProjectCache.from_s3_cache(cache_dir=my_cache_dir)

behavior_session_table = bc.get_behavior_session_table()  
ophys_session_table = bc.get_ophys_session_table()   
experiment_table = bc.get_ophys_experiment_table()                         

#print number of items in each table for all imaging and behavioral sessions
print('Number of behavior sessions = {}'.format(len(behavior_session_table)))
print('Number of ophys sessions = {}'.format(len(ophys_session_table)))
print('Number of ophys experiments = {}'.format(len(experiment_table)))


#print number of items in each table with Mesoscope imaging
print('Number of behavior sessions with Mesoscope = {}'.format(len(behavior_session_table[behavior_session_table.project_code.isin(['VisualBehaviorMultiscope'])])))
print('Number of ophys sessions with Mesoscope = {}'.format(len(ophys_session_table[ophys_session_table.project_code.isin(['VisualBehaviorMultiscope'])])))
print('Number of ophys experiments with Mesoscope = {}'.format(len(experiment_table[experiment_table.project_code.isin(['VisualBehaviorMultiscope'])])))



Number of behavior sessions = 4787
Number of ophys sessions = 705
Number of ophys experiments = 1941
Number of behavior sessions with Mesoscope = 164
Number of ophys sessions with Mesoscope = 164
Number of ophys experiments with Mesoscope = 867


- **Experiment table** contains ophys experiment ids as well as associated metadata with them (```cre_line```, ```session_type```, ```project_code```, etc). This table gives you an overview of what data at the level of each experiment is available. The term *experiment* is used to describe one imaging plane during one session. For sessions that are imaged using mesoscope (```equipment_name``` = *MESO.1*), there will be up to 4 experiments associated with that sessions (2 imaging depths by 2 visual areas). Across sessions, the same imaging planes or experiments are linked using ```ophys_container_id```. For sessions that are imaged using scientifica (```equipment_name``` = *CAMP#.#*), there will be only 1 experiment which are similarly linked across different session types using ```ophys_container_id```.  
- **Ophys session table** is similar to experiment table but it is a higher level overview of the data details. It groups imaging sessions using ```ophys_session_id``` and provides metadata associated with those sessions. 
- **Behavior session table** contains metadata related to animals' training history as well as their behavior during ophys imaging sessions. The table is organized using ```behavior_session_id```. Behavior sessions that were also imaging sessions have ophys ids assosiated with them. 


In this notebook, we will use ```experiment_table``` to select experiments of interest and look at them in a greater detail.

In [7]:
# let's print a sample of 5 rows to see what's in the table
experiment_table.sample(5)


,equipment_name,full_genotype,mouse_id,reporter_line,driver_line,sex,age_in_days,cre_line,indicator,session_number,...,ophys_container_id,project_code,imaging_depth,targeted_structure,date_of_acquisition,session_type,experience_level,passive,image_set,file_id
ophys_experiment_id,,,,,,,,,,,,,,,,,,,,,
994082663,MESO.1,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,485689,Ai148(TIT2L-GC6f-ICL-tTA2),[Sst-IRES-Cre],M,151.0,Sst-IRES-Cre,GCaMP6f,4,...,1018028267,VisualBehaviorMultiscope4areasx2d,273,VISal,2019-12-12 08:34:51.428248+00:00,OPHYS_4_images_H,Novel 1,False,H,1120141649
1051249029,MESO.1,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,523922,Ai148(TIT2L-GC6f-ICL-tTA2),[Vip-IRES-Cre],M,185.0,Vip-IRES-Cre,GCaMP6f,6,...,1046576787,VisualBehaviorMultiscope,140,VISl,2020-09-17 08:24:21.654082+00:00,OPHYS_6_images_B,Novel >1,False,B,1085395919
938002077,MESO.1,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,453988,Ai148(TIT2L-GC6f-ICL-tTA2),[Vip-IRES-Cre],M,207.0,Vip-IRES-Cre,GCaMP6f,3,...,1026722854,VisualBehaviorMultiscope,73,VISp,2019-09-03 08:13:17.884589+00:00,OPHYS_3_images_A,Familiar,False,A,1085400960
1008408514,MESO.1,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,499478,Ai148(TIT2L-GC6f-ICL-tTA2),[Vip-IRES-Cre],F,134.0,Vip-IRES-Cre,GCaMP6f,1,...,1018027819,VisualBehaviorMultiscope4areasx2d,267,VISam,2020-02-18 08:31:29.700735+00:00,OPHYS_1_images_G,Familiar,False,G,1120141977
1039384490,MESO.1,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,513630,Ai93(TITL-GCaMP6f),"[Slc17a7-IRES2-Cre, Camk2a-tTA]",F,202.0,Slc17a7-IRES2-Cre,GCaMP6f,2,...,1037672798,VisualBehaviorMultiscope4areasx2d,275,VISal,2020-07-29 11:32:52.641039+00:00,OPHYS_2_images_G_passive,Familiar,True,G,1120142333


You can get any experiment ids from the experiment table by subsetting the table using various conditions (aka columns) in the table. Here, we can select experiments from Sst mice only, novel Ophys session 4, with 0 prior exposures to the stimulus (meaning the session was not a relake). 

In [8]:
# get all Sst experiments for ophys session 4
selected_experiment_table = experiment_table[(experiment_table.cre_line=='Sst-IRES-Cre')&
                        (experiment_table.session_number==4) &
                        (experiment_table.prior_exposures_to_image_set==0)]
print('Number of experiments: {}'.format(len(selected_experiment_table)))

Number of experiments: 58


Remember that any given experiment contains data from only one imaging plane. Some of these experiments come from the same imaging session. Here, we can check how many unique imaging sessions are associated with experiments selected above.

In [9]:
print('Number of unique sessions: {}'.format(len(selected_experiment_table['ophys_session_id'].unique())))

Number of unique sessions: 19
